In [37]:
import numpy as _np

from pymodels import si
from pyaccel.optics import get_curlyh, calc_twiss, OpticsException
import pyaccel.tracking as _tracking
import pyaccel.lattice as _lattice

%matplotlib qt5
import matplotlib.pyplot as mplt
import matplotlib.gridspec as mgs

In [131]:
accelerator = si.create_accelerator()
energy_offsets = None
track = True
kwargs = dict()

In [166]:
hmax = _lattice.get_attribute(accelerator, 'hmax')
hmin = _lattice.get_attribute(accelerator, 'hmin')

vcham_sts = accelerator.vchamber_on
rad_sts = accelerator.radiation_on
cav_sts = accelerator.cavity_on

accelerator.radiation_on = False
accelerator.cavity_on = False
accelerator.vchamber_on = False

twi0, *_ = calc_twiss(accelerator)

if energy_offsets is None:
    energy_offsets = _np.linspace(1e-6, 6e-2, 60)

if _np.any(energy_offsets < 0):
    raise ValueError('delta must be a positive vector.')

curh_pos = _np.full((energy_offsets.size, len(accelerator)), _np.inf)
curh_neg = _np.full((energy_offsets.size, len(accelerator)), _np.inf)

# Calculate physical aperture
tune_pos = _np.full((2, energy_offsets.size), _np.nan)
tune_neg = _np.full((2, energy_offsets.size), _np.nan)
ap_phys_pos = _np.zeros(energy_offsets.size)
ap_phys_neg = _np.zeros(energy_offsets.size)
twiss_pos = [None] * energy_offsets.size
twiss_neg = [None] * energy_offsets.size
# positive energies
try:
    for idx, delta in enumerate(energy_offsets):
        twi, *_ = calc_twiss(accelerator, energy_offset=delta)
        if _np.any(_np.isnan(twi[0].betax)):
            raise OpticsException('error')
        tune_pos[0, idx] = twi.mux[-1] / 2 / _np.pi
        tune_pos[1, idx] = twi.muy[-1] / 2 / _np.pi
        twiss_pos[idx] = twi
        dcox = twi.rx - twi0.rx
        dcoxp = twi.px - twi0.px
        curh_pos[idx] = get_curlyh(twi.betax, twi.alphax, dcox, dcoxp)

        apper_loc = _np.minimum(
            (hmax - twi.rx)**2, (hmin + twi.rx)**2)
        ap_phys_pos[idx] = _np.min(apper_loc / twi.betax)
except (OpticsException, _tracking.TrackingException):
    pass

# negative energies
try:
    for idx, delta in enumerate(energy_offsets):
        twi, *_ = calc_twiss(accelerator, energy_offset=-delta)
        if _np.any(_np.isnan(twi[0].betax)):
            raise OpticsException('error')
        tune_neg[0, idx] = twi.mux[-1] / 2 / _np.pi
        tune_neg[1, idx] = twi.muy[-1] / 2 / _np.pi
        twiss_neg[idx] = twi
        dcox = twi.rx - twi0.rx
        dcoxp = twi.px - twi0.px
        curh_neg[idx] = get_curlyh(twi.betax, twi.alphax, dcox, dcoxp)

        apper_loc = _np.minimum(
            (hmax - twi.rx)**2, (hmin + twi.rx)**2)
        ap_phys_neg[idx] = _np.min(apper_loc / twi.betax)
except (OpticsException, _tracking.TrackingException):
    pass

# Considering synchrotron oscillations, negative energy deviations will
# turn into positive ones and vice-versa, so the apperture must be
# symmetric:
ap_phys = _np.minimum(ap_phys_pos, ap_phys_neg)

# Calculate Dynamic Aperture
ap_dyn_pos = _np.full(energy_offsets.shape, _np.inf)
ap_dyn_neg = _np.full(energy_offsets.shape, _np.inf)
if track:
    nturns = kwargs.get('nturns_track', 131)
    curh_track = kwargs.get(
        'curh_track', _np.linspace(0, 4e-6, 30))
    ener_pos = kwargs.get(
        'delta_track_pos', _np.linspace(0.02, energy_offsets.max(), 20))
    ener_neg = kwargs.get('delta_track_neg', -ener_pos)

    # Find de 4D orbit to track around it:
    rin_pos = _np.full((6, ener_pos.size, curh_track.size), _np.nan)
    try:
        for idx, en in enumerate(ener_pos):
            rin_pos[:4, idx, :] = _tracking.find_orbit4(
                accelerator, energy_offset=en).ravel()[:, None]
    except _tracking.TrackingException:
        pass
    rin_pos = rin_pos.reshape(6, -1)

    rin_neg = _np.full((6, ener_neg.size, curh_track.size), _np.nan)
    try:
        for idx, en in enumerate(ener_neg):
            rin_neg[:4, idx, :] = _tracking.find_orbit4(
                accelerator, energy_offset=en).ravel()[:, None]
    except _tracking.TrackingException:
        pass
    rin_neg = rin_neg.reshape(6, -1)

    # Get beta at tracking energies to define initial tracking angle:
    beta_pos = _np.ones(energy_offsets.size)
    beta_neg = beta_pos.copy()
    for idx, (twip, twin) in enumerate(zip(twiss_pos, twiss_neg)):
        if twip is not None:
            beta_pos[idx] = twip[0].betax
        if twin is not None:
            beta_neg[idx] = twin[0].betax
    beta_pos = _np.interp(ener_pos, energy_offsets, beta_pos)
    beta_neg = _np.interp(-ener_neg, energy_offsets, beta_neg)

    accelerator.cavity_on = True
    accelerator.radiation_on = True
    accelerator.vchamber_on = True
    orb6d = _tracking.find_orbit6(accelerator)

    # Track positive energies
    curh0, ener = _np.meshgrid(curh_track, ener_pos)
    xl = _np.sqrt(curh0/beta_pos[:, None])

    rin_pos[1, :] += xl.ravel()
    rin_pos[2, :] += 1e-6
    rin_pos[4, :] = orb6d[4] + ener.ravel()
    rin_pos[5, :] = orb6d[5]

    _, _, lostturn_pos, *_ = _tracking.ring_pass(
        accelerator, rin_pos, nturns, turn_by_turn=False)
    lostturn_pos = _np.reshape(lostturn_pos, curh0.shape)
    lost_pos = lostturn_pos != nturns

    ind_dyn = _np.argmax(lost_pos, axis=1)
    ap_dyn_pos = curh_track[ind_dyn]
    ap_dyn_pos = _np.interp(energy_offsets, ener_pos, ap_dyn_pos)

    # Track negative energies:
    curh0, ener = _np.meshgrid(curh_track, ener_neg)
    xl = _np.sqrt(curh0/beta_neg[:, None])

    rin_neg[1, :] += xl.ravel()
    rin_neg[2, :] += 1e-6
    rin_neg[4, :] = orb6d[4] + ener.ravel()
    rin_neg[5, :] = orb6d[5]

    _, _, lostturn_neg, *_ = _tracking.ring_pass(
        accelerator, rin_neg, nturns, turn_by_turn=False)
    lostturn_neg = _np.reshape(lostturn_neg, curh0.shape)
    lost_neg = lostturn_neg != nturns

    ind_dyn = _np.argmax(lost_neg, axis=1)
    ap_dyn_neg = curh_track[ind_dyn]
    ap_dyn_neg = _np.interp(energy_offsets, -ener_neg, ap_dyn_neg)

# Calculate Aperture and Acceptance
ap_dyn_pos = _np.minimum(ap_dyn_pos, ap_phys)
for idx in _np.arange(1, ap_dyn_pos.size):
    ap_dyn_pos[idx] = _np.minimum(ap_dyn_pos[idx], ap_dyn_pos[idx-1])

ap_dyn_neg = _np.minimum(ap_dyn_neg, ap_phys)
for idx in _np.arange(1, ap_dyn_neg.size):
    ap_dyn_neg[idx] = _np.minimum(ap_dyn_neg[idx], ap_dyn_neg[idx-1])

# return curh_pos, curh_neg
comp = curh_pos[:, :] >= ap_dyn_pos[:, None]
idcs = _np.argmax(comp, axis=0)
boo = _np.take_along_axis(comp, _np.expand_dims(idcs, axis=0), axis=0)
idcs[~boo.ravel()] = ap_dyn_pos.size-1
accep_pos = energy_offsets[idcs]

comp = curh_neg[:, :] >= ap_dyn_neg[:, None]
idcs = _np.argmax(comp, axis=0)
boo = _np.take_along_axis(comp, _np.expand_dims(idcs, axis=0), axis=0)
idcs[~boo.ravel()] = ap_dyn_pos.size-1
accep_neg = -energy_offsets[idcs]

accelerator.vchamber_on = vcham_sts
accelerator.radiation_on = rad_sts
accelerator.cavity_on = cav_sts

# return accep_pos, accep_neg

In [168]:
mplt.plot(twi0.spos % (518.4/5), accep_neg)
mplt.plot(twi0.spos % (518.4/5), accep_pos)